# Capítulo 3 - Uso de Requests para consumir APIs
___
Para consumir servicios API REST (los que devuelven un json) instalamos la libreía `requests`:

In [ ]:
! pip install requests

la importamos:

In [ ]:
import requests
import json

## GET
___
consumir un servicio REST dada su url:

In [ ]:
url = "https://aws.random.cat/meow"
response = requests.get(url)

print('STATUS:  ', response.status_code)
print('RESPONSE:', response.text)  # json

data = response.json()  # equivale a: json.loads(response.text)
print('DICT:    ', data)

url = data['file']
print('URL:     ', url)
print()


from IPython.display import Image  # https://ipython.org/ipython-doc/dev/api/generated/IPython.display.html
Image(url, width=300, height=300)


hacer un GET pasando parámetros:

In [ ]:
# url = "https://en.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&exintro=&explaintext=&titles=pizza"  # Bad
# url = "https://en.wikipedia.org/w/api.php"                                                                            # Good

url = "https://test.wikipedia.org//w/api.php"  # mejor usar el entono de pruebas de wikipedia
params = {
    'format': 'json',
    'action': 'query',
    'prop': 'extracts',
    'exintro': '',
    'explaintext': '',
    'titles': 'Page550', 
}

response = requests.get(url, params=params)
print('1.', response.json())
print('2.', response.json()['query'])
print('3.', response.json()['query']['pages'])
print('4.', response.json()['query']['pages'].values())
print('5.', list(response.json()['query']['pages'].values()))
print('6.', list(response.json()['query']['pages'].values())[0])
print('7.', list(response.json()['query']['pages'].values())[0]['extract'])


from IPython.display import JSON  # https://ipython.org/ipython-doc/dev/api/generated/IPython.display.html
JSON(response.json())

Algunos servicios requieren headers o autorización en la petición:

In [ ]:
url = "http://dummy.restapiexample.com/api/v1/employees" 
headers = {'User-Agent': 'PostmanRuntime/7.28.4'}
auth = ('<user>', '<pass>')
response = requests.get(url, headers=headers, auth=auth)

print(response.text)

## POST
___

Se utiliza POST para enviar un documento (generalmente json) junto con la url:

In [ ]:
url = "http://dummy.restapiexample.com/api/v1/create" 
auth = ('<user>', '<pass>')

data = '{"name":"test","salary":123,"age":23}'  # json
headers = {'User-Agent': 'PostmanRuntime/7.28.4',
           'Content-Type': 'application/json'}
response = requests.post(url, headers=headers, auth=auth, data=data)

print('STATUS:  ', response.status_code)
print('RESPONSE:', response.text)

# 
data = {'name': 'test', 'salary': 123, 'age': 23}  # dict
headers = {'User-Agent': 'PostmanRuntime/7.28.4'}
response = requests.post(url, headers=headers, auth=auth, json=data)

print('STATUS:  ', response.status_code)
print('RESPONSE:', response.text)

Algunos servicios requieren headers o autorización en la llamada:

## API SOAP
___
Para consumir servicios API SOAP (los que devuelven un xml) instalamos la libreía `zeep`:

In [ ]:
! pip install zeep

examinar un endpoint:

In [ ]:
! python -mzeep http://www.dneonline.com/calculator.asmx?wsdl

In [ ]:
from zeep import Client

client = Client('http://www.dneonline.com/calculator.asmx?wsdl')
result = client.service.Add(2, 1)  # 2 + 1
print('1:', result)

result = client.service.Subtract(intA=2, intB=1)  # 2 - 1
print('2:', result)

result = client.service['Multiply'](intA=2, intB=1)  # 2 * 1
print('3:', result)

si quiero saber que xml estoy enviando realmente:

In [ ]:
request = client.create_message(client.service, 'Add', intA=2, intB=1)

from zeep.wsdl.utils import etree_to_string
request_xml = etree_to_string(request).decode()

print(request_xml.replace('><', '>\n<'))  # para que se vea bonito

si se quiere el xml tal cual lo responde el servicio:

In [ ]:
with client.settings(raw_response=True):
    result = client.service.Add(2, 1)
    
result_xml = result.text

print(result_xml.replace('><', '>\n<'))  # para que se vea bonito